# Lab Exercises

In this set of exercises, we will use PyGMT to plot global climate datasets.


In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the workshop directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/"))

## NOAA-CIRES 20th Century Reanalysis (V2c)

The first data set is weather maps from [NOAA-CIRES 20th Century Reanalysis](https://psl.noaa.gov/data/gridded/data.20thC_ReanV2c.html). We will choose to plot the air temperature on 31 Dec 2014 on top of the global map.  

Let's load the packages.

In [ ]:
import pygmt
import xarray as xr

Load the NetCDF dataset using xarray

In [ ]:
data = xr.open_dataset('../UTAS_DATA/air.2001.nc')

Again, we first inpsect the data layout.

In [ ]:
data

In [ ]:
data.data_vars

In [ ]:
data.air.dims

In [ ]:
data.time_bnds.dims

In [ ]:
data.air.coords

Now we can inspect the data. 

In [ ]:
data.time_bnds.coords

Alternatively, inspect the data with:

Extract the last time step to plot (e.g., the last time index)

In [ ]:
air_temperature = data['air'].sel(time=data['time'][-1]) 

# Regularise the grid by interpolating to evenly spaced latitude/longitude
lat = air_temperature['lat']
lon = air_temperature['lon']
air_temperature_regular = air_temperature.interp(
    lat=xr.DataArray(sorted(lat.values), dims='lat'),
    lon=xr.DataArray(sorted(lon.values), dims='lon')
)

# Save the regularized data to a temporary NetCDF file
air_temperature_regular.to_netcdf('air_temp_1000mb.nc')

In [ ]:
# Create a PyGMT figure and plot the grid with contours
fig = pygmt.Figure()
fig.grdimage(
    grid='air_temp_1000mb.nc',
    projection='X30c/15c',  
    cmap='rainbow',         
    shading=True           # Add shading for depth perception
)

# Add contour lines on top of the grid
fig.grdcontour(
    grid='air_temp_1000mb.nc',
    interval=2,            # Adjust contour interval for better detail
    pen="0.75p,black"      # Line thickness and color
)

# Add a color bar
fig.colorbar(
    frame='af+l"Temperature (K)"',
    position="JBC+w12c/0.5c+o0c/1c"  # Adjust position and size of color bar as needed
)

# Add the title and labels
fig.basemap(frame=["xa30", "ya30", "+tNOAA-CIRES 20th Century Reanalysis: 1000 mb Air Temperature"])

# Display the figure
fig.show()

In the NOAA-CIRES example, we plotted the data uniformly across the global map. What if the dataset is regional? For example, the oceanic data that are not defined on the land. Let's continue more practice with the NCEP Global Ocean Data Assimilation System (GODAS).

## NCEP Global Ocean Data Assimilation System (GODAS)

In [ ]:
data = xr.open_dataset('../UTAS_DATA/pottmp.2023.nc')  # Replace with your NetCDF file path

In [ ]:
# Example: Get the first 10 entries along each dimension
data_point = data['pottmp'].isel(time=-1, level=5)
print(data_point)
print(data_point.values)

In [ ]:
# Regularise the grid by interpolating to uniform latitude/longitude spacing
pottmp_regular = pottmp.interp(
    lat=xr.DataArray(pottmp['DATA_VARIABLE1']), # inspect the dataset and replace DATA_VARIABLE1, DATA_VARIABLE2
    lon=xr.DataArray(pottmp['DATA_VARIABLE2'])
)

# Check data range
pottmp_min = pottmp_regular.min().values
pottmp_max = pottmp_regular.max().values
print("Data range:", pottmp_min, pottmp_max)

# Save the grid data to a temporary NetCDF file
pottmp_regular.to_netcdf('pottmp_5m_regular.nc')

In [ ]:

# Create a PyGMT figure
fig = pygmt.Figure()

fig.basemap(region=[0, 360, -90, 90], projection="X15c/7c", frame=True)

fig.coast(region=[0, 360, -90, 90], projection="X15c/7c", land="white", water="lightblue", frame="af")

# Generate the color palette within the figure's context
pygmt.makecpt(
    cmap="rainbow",
    series=[pottmp_min, pottmp_max]  # Set min and max values for the color scale
)

# Plot the temperature data using grdimage
fig.grdimage(
    grid='pottmp_5m_regular.nc',
    cmap=True,               # Use the most recent CPT created by makecpt
    shading=True,
    nan_transparent=True     # Make NaN values transparent
)

# Optionally add contour lines for clarity
fig.grdcontour(
    grid='pottmp_5m_regular.nc',
    interval=2,
    pen="0.5p,black"
)

# Add a color bar to show the temperature scale
fig.colorbar(
    frame="af+l'Temperature (K)'",
    position="JBC+w10c/0.5c"
)

# Display the plot
fig.show()
print("Minimum temperature value:", pottmp_min)